We are calculating variances and covariances as:

$$
var(\hat{y}^i_{h,t}) = \big[\sigma^2_{\alpha} + 2 \sigma_{\alpha \beta} h + \sigma^2_{\beta}h^2 \big] + var(z_{h,t}^i) +  \phi_t^2 \sigma^2_{\epsilon}
$$


$$
covar(\hat{y}^i_{h,t}, \hat{y}^i_{h+n,t+n}) = \big[\sigma^2_{\alpha} + 2 \sigma_{\alpha \beta} (h+n) + \sigma^2_{\beta}h(h+n) \big] + \rho^2 var(z_{h,t}^i) 
$$

Where $n = 1,...,min(H-h, T-t)$ is the minimum of the distance of an individual to the maximum experience and the maximum time period - e.g. an individual who is age 20 in 1995 could potentially have 45 years of experience, but as the sample is truncated in 1996, $n$ will be 1 ($T-t$), while an individual who is 60 years old in 1970 could stay in the sample for 26 more years, but as age is capped at 64, $n$ will be 4 ($H-h$).

The variance of $z_{h,t}^i$ will be obtained recursively by:

\begin{align}
var(z_{1,t}^i) &= \pi^2_t \sigma^2_{\eta}, \ h=1, t>1 \\
var(z_{h,1}^i) &= \pi^2_1 \sigma^2_{\eta} \sum_{j=0}^{h-1}\rho^{2j}, \ t=1, h>1 \\
var(z_{h,t}^i) &= \rho^2 var(z_{h-1,t-1}) + \pi_t^2 \sigma^2_{\eta}, \ t,h>1
\end{align}

Where the first line implicitly assumes that the initial value of the persistent shock is zero for all individuals (i.e. at $h=1$, the first period of working life, there is no $\rho z_0$ entering the individuals income process).

What the above equations tell us is that we will have two slightly different ways of calculating an individual's variance for any $t,h>1$, depending on whether $h \leq t$ or $h > t$. Intuitively, this is due to the fact that when $h > t$, the individual has already accumulated a history of shocks at $t=1$, the first period of observation. In this case, the recursion for $var(z_{h,t}^i)$ will end with the $var(z_{h,1}^i)$ term, so individuals will differ based on how many working years they accrued before the start of the sample (these are individuals who started work prior to 1968). If $h \leq t$, the recursion ends in $var(z_{1,t}^i)$, so individuals will only differ in the first shock they experienced upon entering the sample at the start of their working life (these are individuals who start work after 1968).